# NHL Models

## Data Export
```
# skater data
dumpdata.sc --seasons 20152016 20162017 20172018 20182019 20192020 20202021 20212022 --stats "*" --progress \
    nhl_hist_20072008-20212022.scored.db --no_teams --target_calc_stats "*" \
    --pos LW RW W C D \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home -f nhl_skater.csv

# goalie data
dumpdata.sc --seasons 20152016 20162017 20172018 20182019 20192020 20202021 20212022 --stats "*" --progress \
    nhl_hist_20072008-20212022.scored.db --no_teams --target_calc_stats "*" \
    --pos G \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home -f nhl_goalie.csv

# team data
dumpdata.sc --seasons 20152016 20162017 20172018 20182019 20192020 20202021 20212022 --stats "*" --progress \
    nhl_hist_20072008-20212022.scored.db --no_players --target_stats goal win \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home -f nhl_team.csv
```

In [ ]:
RANDOM_SEED = 1
VALIDATION_SEASON = 20212022
RECENT_GAMES = 5
TRAINING_SEASONS = [
    20152016,
    20162017,
    20172018,
    20182019,
    20192020,
    20202021,
]


In [ ]:
# Model Creation
from datetime import datetime

from fantasy_py import PlayerOrTeam
from train_test import load_data, train_test, create_fantasy_model

# TARGET = ("stat", "goal")
# MODEL_NAME = "NHL-team-goal"
# DATA_FILENAME = "/fantasy/nhl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None
# TRAINING_TIME = 600

# TARGET = ("stat", "win")
# MODEL_NAME = "NHL-team-win"
# DATA_FILENAME = "/fantasy/nhl_team.csv"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TARGET_POS = None
# TRAINING_TIME = 600

TARGET = ("calc", "dk_score")
MODEL_NAME = "NHL-skater-DK"
DATA_FILENAME = "/fantasy/nhl_skater.csv"
INCLUDE_POS = True
P_OR_T = PlayerOrTeam.PLAYER
TARGET_POS = ["LW", "RW", "W", "C", "D"]
TRAINING_TIME = 900

# TARGET = ("calc", "dk_score")
# MODEL_NAME = "NHL-goalie-DK"
# DATA_FILENAME = "/fantasy/nhl_goalie.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = False
# TARGET_POS = ["G"]
# TRAINING_TIME = 600

raw_df, tt_data = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    include_position=INCLUDE_POS,
    seed=RANDOM_SEED,
)

for automl_type in ["tpot"]:  # , "autosk"]:
    dt_trained = datetime.now()

    model_filepath, performance = train_test(
        automl_type, MODEL_NAME, TARGET, tt_data, RANDOM_SEED, TRAINING_TIME, dt_trained
    )
    performance["season"] = VALIDATION_SEASON
    model = create_fantasy_model(
        MODEL_NAME,
        model_filepath,
        dt_trained,
        tt_data[0],
        TARGET,
        TRAINING_TIME,
        P_OR_T,
        RECENT_GAMES,
        automl_type,
        performance,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
        target_pos=TARGET_POS,
        training_pos=TARGET_POS,
        raw_df=raw_df
    )
    model_filepath = model.dump(
        ".".join([MODEL_NAME, TARGET[1], automl_type, "model"]),
        validate_reg_filepath=False,
    )
    print(f"Model file saved to '{model_filepath}'")
